Boston Housing Dataset Test
===========================

In this notebook we test revrand's ARD basis functions on the Boston housing dataset.

In [28]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.cross_validation import KFold
from sklearn.metrics import r2_score

from revrand import StandardLinearModel
import revrand.basis_functions as bf
from revrand.metrics import smse, msll
from revrand.btypes import Parameter, Positive, Bound

# Log output to the terminal attached to this notebook
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")


In [29]:
# Load the data
boston = load_boston()
X = boston.data
y = boston.target - boston.target.mean()

N, D = X.shape
lenscale = 10.
nbases = 400
lenARD = lenscale * np.ones(D)


In [30]:
# Construct basis functions
base = bf.RandomMatern32(Xdim=D, nbases=nbases, lenscale_init=Parameter(lenARD, Positive())) + \
    bf.LinearBasis(onescol=True)
#base = bf.spectralmixture(Xdim=D, bases_per_component=5, lenscales_init=[Parameter(l, Positive()) for l in lenARD]) + \
#    bf.LinearBasis(onescol=True)
#base = bf.SigmoidalBasis(centres=X[np.random.choice(N, 10), :], lenscale_init=Parameter(lenARD, Positive())) + \
#    bf.LinearBasis(onescol=True)
#base = bf.FastFoodRBF(Xdim=D, nbases=nbases, lenscale_init=Parameter(lenARD, Positive())) + \
#    bf.LinearBasis(onescol=True)
    

In [31]:
# Cross val
folds = 5
av_smse, av_msll, av_r2 = 0., 0., 0.

slm = StandardLinearModel(base)

for i, (tr_ind, ts_ind) in enumerate(KFold(len(y), n_folds=folds, shuffle=True)):
    
    # Training
    slm.fit(X[tr_ind], y[tr_ind])
    
    # Prediction
    Ey, Vy = slm.predict_moments(X[ts_ind])

    # Validation
    f_smse = smse(y[ts_ind], Ey)
    f_msll = msll(y[ts_ind], Ey, Vy, y[tr_ind])
    f_r2 = r2_score(y[ts_ind], Ey)
    
    av_smse += f_smse
    av_msll += f_msll
    av_r2 += f_r2
    
    
    print("Fold: {},\n\trevrand: R2 = {}, SMSE = {}, MSLL = {}".format(i, f_r2, f_smse, f_msll))
    
av_smse /= folds
av_msll /= folds
av_r2 /= folds

Fold: 0,
	revrand: R2 = 0.7903753490882924, SMSE = 0.2096246509117076, MSLL = -0.6111224653206332
Fold: 1,
	revrand: R2 = 0.7794265338728856, SMSE = 0.2205734661271144, MSLL = -0.7396646691336631
Fold: 2,
	revrand: R2 = 0.7868480723799158, SMSE = 0.21315192762008414, MSLL = -0.8065068397239648
Fold: 3,
	revrand: R2 = 0.7011603786224245, SMSE = 0.29883962137757547, MSLL = -0.6924375737240718
Fold: 4,
	revrand: R2 = 0.798786078102757, SMSE = 0.20121392189724296, MSLL = -0.8547457020584834


In [32]:
# Print results
print("Final:\n\trevrand: R2 = {}, SMSE = {}, MSLL = {}".format(av_r2, av_smse, av_msll))


Final:
	revrand: R2 = 0.771319282413255, SMSE = 0.2286807175867449, MSLL = -0.7408954499921633
